In [172]:
import geopandas as gpd
import glob
import os

In [174]:
# 0: Geodataframe creation
def g(geom, crs):
    try:
        df = gpd.GeoDataFrame(geometry=geom, crs=crs)
        return df
    except:
        df = gpd.GeoDataFrame(geometry=[geom], crs=crs)
        return df


In [298]:
""" Dallas Edges Buffer """
dal = gpd.read_file('edges_48113.shp')
dal = dal[['geometry']]
dal.reset_index(inplace=True)

# Buffer each polygon as new geometry
dal['buff'] = dal.apply(lambda x: x.geometry.buffer(5), axis=1)
dal_buff = dal[['index', 'buff']].set_geometry('buff', crs=26913)
dal_buff.head()



,index,buff
0,0,"POLYGON ((1245430.016 3677717.883, 1245431.363..."
1,1,"POLYGON ((1245620.718 3677496.475, 1245619.371..."
2,2,"POLYGON ((1245619.256 3675195.835, 1245619.164..."
3,3,"POLYGON ((1245612.125 3675236.397, 1245611.398..."
4,4,"POLYGON ((1245643.427 3675106.022, 1245643.902..."


In [299]:
""" Test single edge and intersect with County Boundary """
df = g(dal_buff.iloc[10].buff, 26913)
counties = gpd.read_file('counties.shp')
counties = counties.to_crs(26913)
dal = counties.loc[counties['FIPSSTCO'] == '48113']



In [311]:
""" Tar test """
tar = gpd.read_file('edges_48439.shp')
tar = tar[['geometry']]
tar.reset_index(inplace=True, drop=True)
#tar = g(tar.iloc[51].geometry, 26913)


In [314]:
tar.geometry.sindex.nearest(df.geometry)

array([[ 0,  0],
       [52, 51]], dtype=int64)

In [300]:
""" Dallas Buffer Intersection with Tarrant County """
tar = gpd.read_file('edges_48439.shp')
tar = tar[['geometry']]
tar.reset_index(inplace=True, drop=True)

# New intersection geometry in buff gdf
dal_intersect = dal_buff.sjoin(tar)
tar.reset_index(inplace=True)
dal_intersect = dal_intersect.merge(tar, left_on='index_right', right_on='index')

dal_intersect




,index_x,buff,index_right,index_y,geometry
0,6,"POLYGON ((1246536.701 3661194.647, 1246537.211...",53,53,"POLYGON ((1246365.219 3661102.333, 1246366.859..."
1,7,"POLYGON ((1246555.528 3661225.763, 1246555.018...",53,53,"POLYGON ((1246365.219 3661102.333, 1246366.859..."
2,6,"POLYGON ((1246536.701 3661194.647, 1246537.211...",56,56,"POLYGON ((1246518.549 3661050.830, 1246519.592..."
3,8,"POLYGON ((1246543.653 3661099.773, 1246544.725...",56,56,"POLYGON ((1246518.549 3661050.830, 1246519.592..."
4,6,"POLYGON ((1246536.701 3661194.647, 1246537.211...",55,55,"POLYGON ((1246499.049 3661088.255, 1246499.864..."
5,7,"POLYGON ((1246555.528 3661225.763, 1246555.018...",55,55,"POLYGON ((1246499.049 3661088.255, 1246499.864..."
6,8,"POLYGON ((1246543.653 3661099.773, 1246544.725...",55,55,"POLYGON ((1246499.049 3661088.255, 1246499.864..."
7,9,"POLYGON ((1246781.021 3657183.339, 1246782.943...",54,54,"POLYGON ((1246475.109 3656782.062, 1246477.049..."
8,11,"POLYGON ((1246800.786 3656854.249, 1246804.742...",54,54,"POLYGON ((1246475.109 3656782.062, 1246477.049..."
9,9,"POLYGON ((1246781.021 3657183.339, 1246782.943...",52,52,"POLYGON ((1246309.844 3657133.079, 1246311.688..."


In [295]:
dal_intersect

,index_x,buff,index_right,index_y,geometry,union_geom
0,6,"POLYGON ((1246536.701 3661194.647, 1246537.211...",53,53,"POLYGON ((1246365.219 3661102.333, 1246366.859...","POLYGON ((1246540.270 3661207.768, 1246540.780..."
1,7,"POLYGON ((1246555.528 3661225.763, 1246555.018...",53,53,"POLYGON ((1246365.219 3661102.333, 1246366.859...","POLYGON ((1246555.018 3661223.889, 1246554.508..."
2,6,"POLYGON ((1246536.701 3661194.647, 1246537.211...",56,56,"POLYGON ((1246518.549 3661050.830, 1246519.592...","POLYGON ((1246537.211 3661196.521, 1246537.721..."
3,8,"POLYGON ((1246543.653 3661099.773, 1246544.725...",56,56,"POLYGON ((1246518.549 3661050.830, 1246519.592...","POLYGON ((1246544.725 3661101.448, 1246545.797..."
4,6,"POLYGON ((1246536.701 3661194.647, 1246537.211...",55,55,"POLYGON ((1246499.049 3661088.255, 1246499.864...","POLYGON ((1246537.211 3661196.521, 1246537.721..."
5,7,"POLYGON ((1246555.528 3661225.763, 1246555.018...",55,55,"POLYGON ((1246499.049 3661088.255, 1246499.864...","POLYGON ((1246555.018 3661223.889, 1246554.508..."
6,8,"POLYGON ((1246543.653 3661099.773, 1246544.725...",55,55,"POLYGON ((1246499.049 3661088.255, 1246499.864...","POLYGON ((1246551.156 3661111.500, 1246552.227..."
7,9,"POLYGON ((1246781.021 3657183.339, 1246782.943...",54,54,"POLYGON ((1246475.109 3656782.062, 1246477.049...","POLYGON ((1246782.943 3657183.723, 1246786.786..."
8,11,"POLYGON ((1246800.786 3656854.249, 1246804.742...",54,54,"POLYGON ((1246475.109 3656782.062, 1246477.049...","POLYGON ((1246804.742 3656854.676, 1246806.720..."
9,9,"POLYGON ((1246781.021 3657183.339, 1246782.943...",52,52,"POLYGON ((1246309.844 3657133.079, 1246311.688...","POLYGON ((1246782.943 3657183.723, 1246786.786..."


In [297]:
# Clean up gdf
dal_intersect = dal_intersect[['index_x', 'union_geom']]
dal_intersect.rename(columns={'union_geom': 'geometry'}, inplace=True)
dal_intersect = dal_intersect.set_geometry('geometry', crs=26913)
dal_intersect.to_file('dal_union.shp')


C:\Users\abreunig\AppData\Local\Temp\2\ipykernel_6668\4293747054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dal_intersect.rename(columns={'union_geom': 'geometry'}, inplace=True)


In [228]:
dal_union.to_file('dal_union.shp')